# Colorize BW Images

In [102]:
import numpy as np 
import cv2
from cv2 import dnn

### MODEL FILE PATHS

In [103]:
proto_file = './model/colorization_deploy_v2.prototxt' 
model_file = './model/colorization_release_v2.caffemodel'
hull_pts = './model/pts_in_hull.npy'
img_path = './images/mariaBonita.webp'

### READING THE MODEL PARAMS

In [104]:
net = dnn.readNetFromCaffe(proto_file, model_file)
kernel = np.load(hull_pts)

### READING AND PREPROCESSING IMAGE

In [105]:
img = cv2.imread(img_path)
scaled = img.astype("float32") / 255.0
lab_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

### ADD THE CLUSTER CENTERS AS 1X1 CONVOLUTIONS TO THE MODEL

In [106]:
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

### RESIZE THE IMAGE FOR THE NETWORK

In [107]:
resized = cv2.resize(lab_img, (224, 224))
# split the L channel
L = cv2.split(resized)[0]
L -= 50

### PREDICTING THE AB CHANNELS FROM THE INPUT L CHANNEL

In [108]:
net.setInput(cv2.dnn.blobFromImage(L))
ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))

### RESIZE THE PREDICTED 'AB' VOLUME TO THE SAME DIMENSIONS AS OUR INPUT IMAGE

In [109]:
ab_channel = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))

### TAKE THE L CHANNEL FROM THE IMAGE

In [110]:
L = cv2.split(lab_img)[0]

### JOIN THE L CHANNEL WITH PREDICTED AB CHANNEL

In [111]:
colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

### THEN CONVERT THE IMAGE FROM LAB TO BGR 

In [112]:
# colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2RGB) # Da la tonalidad azul
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

colorized = (255 * colorized).astype("uint8")

img = cv2.resize(img,(640,640))
colorized = cv2.resize(colorized,(640,640))

result = cv2.hconcat([img,colorized])

""" cv2.imshow("BW Image", img)
cv2.imshow("Colorizes",colorized) """

cv2.imshow("Grayscale -> Colour", result)
colorized = cv2.resize(colorized,(1600,900))

cv2.imwrite('./results/colorized_maria.png', colorized)

cv2.waitKey(0)

-1